<a href="https://colab.research.google.com/github/VenkataViswas/A_MTech-Sem2-Lab-Programs/blob/main/DL/Deep_Learning_lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:

# Dataset
dataset = np.array([
    [2, 2, 0],
    [1, 2, 0],
    [5, 4, 1],
    [2, 1, 0],
    [3, 3, 1],
    [7, 2, 1],
    [5, 2, 1],
    [6, 1, 1],
    [8, -0.2, 1],
    [7, 3, 1]
], dtype=float)

X = dataset[:, :-1]
y = dataset[:, -1]


In [ ]:

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



In [ ]:

model = Sequential([
    Dense(1, activation='sigmoid', input_shape=(2,))
])

model.compile(
    optimizer=SGD(learning_rate=0.05),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Training
model.fit(X_train, y_train, epochs=300, verbose=0)

# Evaluation
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 1.00


In [ ]:

# Predictions
pred = (model.predict(X_test) > 0.5).astype(int)
for i in range(len(X_test)):
    print(f"Expected={int(y_test[i])}, Predicted={pred[i][0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Expected=1, Predicted=1
Expected=0, Predicted=0


## Bank Marketing
The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).

In [39]:
!pip install ucimlrepo

In [43]:

import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



In [44]:

# ----------------------------------------------------------
# 1. Load Dataset
# ----------------------------------------------------------

bank_marketing = fetch_ucirepo(id=222)

X = bank_marketing.data.features
y = bank_marketing.data.targets

print("Dataset Loaded!")
print("Shape:", X.shape)



Dataset Loaded!
Shape: (45211, 16)


In [45]:

# ----------------------------------------------------------
# 2. Preprocess Target Variable
# ----------------------------------------------------------

# Target is 'yes' / 'no' -> convert to 1 / 0
y = y['y'].map({'yes': 1, 'no': 0})



In [46]:

# ----------------------------------------------------------
# 3. Identify Column Types
# ----------------------------------------------------------

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

print("\nNumeric Features:", numeric_features.tolist())
print("\nCategorical Features:", categorical_features.tolist())



Numeric Features: ['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']

Categorical Features: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


In [47]:


# ----------------------------------------------------------
# 4. Build Preprocessing Pipeline
# ----------------------------------------------------------

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)



In [48]:

# 5 Train-test split (stratify keeps class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Fit transform train; transform test
X_train_p = preprocessor.fit_transform(X_train)
X_test_p = preprocessor.transform(X_test)

# ColumnTransformer returns sparse matrix when OneHot used
# Keras works best with dense arrays
X_train_p = X_train_p.toarray() if hasattr(X_train_p, "toarray") else np.array(X_train_p)
X_test_p  = X_test_p.toarray()  if hasattr(X_test_p,  "toarray") else np.array(X_test_p)

print("\nAfter preprocessing:")
print("X_train_p shape:", X_train_p.shape)
print("X_test_p shape :", X_test_p.shape)



After preprocessing:
X_train_p shape: (36168, 51)
X_test_p shape : (9043, 51)


In [50]:

from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# 6) Build ANN Model
# -----------------------------
tf.random.set_seed(42)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_p.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# -----------------------------
# 7) Train
# -----------------------------
history = model.fit(
    X_train_p, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

print("\nTraining Complete!")


# -----------------------------
# 8) Predict & Evaluate
# -----------------------------
proba = model.predict(X_test_p).reshape(-1)
y_pred = (proba >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)

print("\n===== MODEL PERFORMANCE (ANN) =====")
print(f"Accuracy: {acc * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


# -----------------------------
# 9) Sample Predictions
# -----------------------------
print("\n===== SAMPLE PREDICTIONS =====")
idx = np.random.choice(len(X_test_p), size=min(10, len(X_test_p)), replace=False)

for i in idx:
    print(f"Actual={int(y_test.iloc[i])}  Predicted={int(y_pred[i])}  Prob={proba[i]:.3f}")

Epoch 1/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8661 - loss: 0.3060 - val_accuracy: 0.9054 - val_loss: 0.2134
Epoch 2/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9024 - loss: 0.2218 - val_accuracy: 0.9053 - val_loss: 0.2103
Epoch 3/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9034 - loss: 0.2143 - val_accuracy: 0.9078 - val_loss: 0.2068
Epoch 4/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9080 - loss: 0.2095 - val_accuracy: 0.9083 - val_loss: 0.2056
Epoch 5/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9069 - loss: 0.2056 - val_accuracy: 0.9096 - val_loss: 0.2036
Epoch 6/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9074 - loss: 0.2028 - val_accuracy: 0.9086 - val_loss: 0.2035
Epoch 7/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9085 - loss: 0.1975 - val_accuracy: 0.9083 - val_loss: 0.2030
Epoch 8/100
905/905 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9096 - loss: 0.1971 - val_accu